# CAS2105 Mini AI Pipeline Project
## pipeline_demo.ipynb

This notebook demonstrates:
1. Load Dataset (IMDB)
2. Naïve keyword-based baseline
3. Transformer-based AI pipeline (DistilBERT)
4. Comparison


## 1. Load Dataset (IMDB)

In [29]:
from datasets import load_dataset
import random

# load imdb dataset get 200
dataset = load_dataset("imdb", split="test").shuffle(seed=42).select(range(200))

# check data once
print(f"Total examples: {len(dataset)}")
print(f"Example: {dataset[2]['text'][:100]}... Label: {dataset[2]['label']}")
# Label 0 = Negative, 1 = Positive

Total examples: 200
Example: This movie was so frustrating. Everything seemed energetic and I was totally prepared to have a good... Label: 0


## 2. Naïve Baseline: Keyword Rules

In [30]:
def naive_baseline(text):
    text = text.lower()
    # define positive word or negative word
    pos_words = ["good", "great", "excellent", "amazing", "love", "best", "wonderful"]
    neg_words = ["bad", "terrible", "awful", "worst", "boring", "hate", "stupid"]

    pos_score = sum(1 for word in pos_words if word in text)
    neg_score = sum(1 for word in neg_words if word in text)

    # discuss positive or negative of text
    if pos_score >= neg_score:
        return 1 # Positive
    else:
        return 0 # Negative

# run dataset on baseline
baseline_preds = [naive_baseline(item['text']) for item in dataset]

## 3. AI Pipeline: DistilBERT

In [31]:
from transformers import pipeline

# load pre-train pipeline
ai_classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

def run_ai_pipeline(data):
    predictions = []
    # only get 512 text token
    texts = [d['text'][:512] for d in data]
    results = ai_classifier(texts)

    for res in results:
        # turn positive or negative to 1 ior 0
        label = 1 if res['label'] == 'POSITIVE' else 0
        predictions.append(label)
    return predictions

ai_preds = run_ai_pipeline(dataset)

Device set to use cpu


## 4. Result Accuracy

In [28]:
from sklearn.metrics import accuracy_score, classification_report

labels = dataset['label']

print("--- Naïve Baseline Results ---")
print(f"Accuracy: {accuracy_score(labels, baseline_preds):.4f}")

print("\n--- AI Pipeline Results ---")
print(f"Accuracy: {accuracy_score(labels, ai_preds):.4f}")

# Failure Analysis
print("\n--- Difference Examples ---")
for i in range(len(dataset)):
    if baseline_preds[i] != ai_preds[i]:
        print(f"Text: {dataset[i]['text'][:100]}...")
        print(f"True Label: {labels[i]}, Baseline: {baseline_preds[i]}, AI: {ai_preds[i]}")
        break

--- Naïve Baseline Results ---
Accuracy: 0.6050

--- AI Pipeline Results ---
Accuracy: 0.7900

--- Difference Examples ---
Text: This movie spends most of its time preaching that it is the script that makes the movie, but apparen...
True Label: 0, Baseline: 1, AI: 0
